# Chem277B: Machine Learning Algorithms

## Homework assignment #5: Regression

In [4]:
import numpy as np 
import pandas as pd
import math 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split, KFold

### 1. Baye’s Theorem.

(a) From the given data, the categories of testing results and their probabilities within proportion have been summarized in the table below:

Category | Probability within Proportion | Kidney Disease Positive | Marker | Proportion
--- | --- | --- | --- | ---
P[+\|M] | 0.95 | $+$ | $+$ | 0.95 * 0.01
P[-\|M] | (1-0.95) | $-$ | $+$ | (1-0.95) * 0.01
P[+\|not M] | (1-0.95) | $+$ | $-$ | (1-0.95) * 0.99
P[-\|not M] | 0.95 | $-$ | $-$ | 0.95 * 0.99

Hence the quantities for the questions are:

(a1) P[-\|M] = (1-0.95) = 0.05 within its proportion, the absolute probability is 0.05 * 0.01 = 0.0005

(a2) P[+\|not M] = (1-0.95) = 0.05 within its proportion, the absolute probability is 0.05 * 0.99 = 0.0495

(a3) P[not M] = (1-0.95) * 0.99 + 0.95 * 0.99 = 0.99, or 1 - 0.01 = 0.99

(b) Using the Baye's Theorem, we try to differentiate between positive marker + positive test and positive marker + negative test. Hence the calculation is defined as:
$$ P[M|+] = \frac{P[+|M] * P[M]}{P[+|M] * P[M] + P[+|not M] * P[not M]} $$   
$$ = \frac{0.95 \times 0.01}{(0.95 \times 0.01) + (0.05 \times 0.99)} = 0.161 $$

Hence the chance of testing positive and actually have the marker is 16.1%. It warrants additional testing to confirm.

(c) When P[M] = 0.10, the categories and probabilities become the following:

Category | Probability within Proportion | Kidney Disease Positive | Marker | Proportion
--- | --- | --- | --- | ---
P[+\|M] | 0.95 | $+$ | $+$ | 0.95 * 0.10
P[-\|M] | (1-0.95) | $-$ | $+$ | (1-0.95) * 0.10
P[+\|not M] | (1-0.95) | $+$ | $-$ | (1-0.95) * 0.90
P[-\|not M] | 0.95 | $-$ | $-$ | 0.95 * 0.90

Hence with the new frequency, the individual who test positive actually has the marker is:
$$ P[M|+]' = \frac{P[+|M]' * P[M]'}{P[+|M]' * P[M]' + P[+|not M]' * P[not M]'} $$   
$$ = \frac{0.95 \times 0.1}{(0.95 \times 0.1) + (0.05 \times 0.9)} = 0.679 $$


### 2. Gaussian Naive Bayes.

(a) The finished codes are shown below.

I chose Gaussian distribution because it's a widely used normal probability distributions in statistics, data analysis and visualization. 

The Gaussian distribution has a bell curve with mean and standard deviation, hence suitable for modeling many real-world phenomena.

With the finished function, I calculated that a wine from cultivar 1 has a 23.24% probability of containinh 13% Alcohol.

In [5]:
class NaiveBayesClassifier():
    def __init__(self):
        self.type_indices={}    # store the indices of wines that belong to each cultivar as a boolean array of length 178
        self.type_stats={}      # store the mean and std of each cultivar
        self.ndata = 0
        self.trained=False
    
    @staticmethod
    def gaussian(x,mean,std):
        exponent = -(x - mean)**2 / (2 * std**2)
        
        return (np.exp(exponent) / (np.sqrt(2 * np.pi) * std))
    
    @staticmethod
    def calculate_statistics(x_values):
        # Returns a list with length of input features. Each element is a tuple, with the input feature's average and standard deviation
        n_feats=x_values.shape[1]
        return [(np.average(x_values[:,n]),np.std(x_values[:,n])) for n in range(n_feats)]
    
    @staticmethod
    def calculate_prob(x_input,stats):
        """Calculate the probability that the input features belong to a specific class(P(X|C)), defined by the statistics of features in that class
        x_input: np.array shape(nfeatures)
        stats: list of tuple [(mean1,std1),(means2,std2),...]
        """ 
        init_prob = 1 
        for i in range(len(x_input)):
            mean, std = stats[i]
            init_prob *= NaiveBayesClassifier.gaussian(x_input[i], mean, std)
        return init_prob
    
    def fit(self,xs,ys):
        # Train the classifier by calculating the statistics of different features in each class
        self.ndata = len(ys)
        for y in set(ys):
            type_filter= (ys==y)
            self.type_indices[y]=type_filter
            self.type_stats[y]=self.calculate_statistics(xs[type_filter])
        self.trained=True
            
    def predict(self,xs):
        # Do the prediction by outputing the class that has highest probability
        if len(xs.shape)>1:
            print("Only accepts one sample at a time!")
        if self.trained:
            guess=None
            max_prob=0
            # P(C|X) = P(X|C)*P(C) / sum_i(P(X|C_i)*P(C_i)) (deniminator for normalization only, can be ignored)
            for y_type in self.type_stats:
                pre = sum(self.type_indices[y_type]) / self.ndata
                prob= self.calculate_prob(xs, self.type_stats[y_type]) * pre
                if prob>max_prob:
                    max_prob=prob
                    guess=y_type
            return guess
        else:
            print("Please train the classifier first!")
            
def calculate_accuracy(model,xs,ys):
    y_pred=np.zeros_like(ys)
    for idx,x in enumerate(xs):
        y_pred[idx]=model.predict(x)
    return np.sum(ys==y_pred)/len(ys)

In [6]:
# Import wines.csv
wines = pd.read_csv('wines.csv')
wines.head()

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline,Start assignment,ranking
0,14.23,1.71,2.43,15.6,127,2.8,3.06,0.28,2.29,5.64,1.04,3.92,1065,1,1
1,13.24,2.59,2.87,21.0,118,2.8,2.69,0.39,1.82,4.32,1.04,2.93,735,1,1
2,14.83,1.64,2.17,14.0,97,2.8,2.98,0.29,1.98,5.20,1.08,2.85,1045,1,1
3,14.12,1.48,2.32,16.8,95,2.2,2.43,0.26,1.57,5.00,1.17,2.82,1280,1,1
4,13.75,1.73,2.41,16.0,89,2.6,2.76,0.29,1.81,5.60,1.15,2.90,1320,1,1


In [7]:
# Define the instance from the Naive Bayes Classifier
nbc = NaiveBayesClassifier()

# Fit the Naive Bayes Classifier
nbc.fit(wines.loc[:, 'Alcohol %':'Proline'].values, wines.loc[:, 'ranking'].values)

# First get the stats for cultivar 1
type_stats = nbc.type_stats[1]

# Then calculate the probability of alcohol% = 13
probability = nbc.gaussian(13, type_stats[0][0], type_stats[0][1])

print(f"A wine from cultivar 1 has a {round(probability*100, 2)}% probability of containinh 13% Alcohol")

A wine from cultivar 1 has a 23.24% probability of containinh 13% Alcohol


(b) After 3-fold training, I can achieve close to 100% accuracy in very short term. The Naive Baye's method performs much better and much faster than the simulated annealing method.

In [8]:
# First normalize the wines dataframe
wines_norm = wines.loc[:, 'Alcohol %':'Proline']
wines_norm = (wines_norm - np.mean(wines_norm, axis=0)) / np.std(wines_norm, axis=0)
wines_norm = wines_norm.merge(wines[['Start assignment','ranking']], left_index=True, right_index=True)
wines_norm.head()

,Alcohol %,Malic Acid,Ash,Alkalinity,Mg,Phenols,Flavanoids,Phenols.1,Proantho-cyanins,Color intensity,Hue,OD280 315,Proline,Start assignment,ranking
0,1.518613,-0.562250,0.232053,-1.169593,1.913905,0.808997,1.034819,-0.659563,1.224884,0.251717,0.362177,1.847920,1.013009,1,1
1,0.295700,0.227694,1.840403,0.451946,1.281985,0.808997,0.663351,0.226796,0.401404,-0.319276,0.362177,0.449601,-0.037874,1,1
2,2.259772,-0.625086,-0.718336,-1.650049,-0.192495,0.808997,0.954502,-0.578985,0.681738,0.061386,0.537671,0.336606,0.949319,1,1
3,1.382733,-0.768712,-0.170035,-0.809251,-0.332922,-0.152402,0.402320,-0.820719,-0.036617,-0.025128,0.932531,0.294232,1.697675,1,1
4,0.925685,-0.544297,0.158946,-1.049479,-0.754202,0.488531,0.733629,-0.578985,0.383884,0.234414,0.844785,0.407228,1.825055,1,1


In [9]:
# Divide the normalized wines data into 3-fold training and testing groups
# and use 2/3 training and 1/3 testing for the three divisions
kf = KFold(n_splits=3, shuffle=True)
xs = wines.loc[:, 'Alcohol %':'Proline'].values 
ys = wines.loc[:, 'ranking'].values
nbc = NaiveBayesClassifier()
accuracy = []

for train_index, test_index in kf.split(xs):
    x_train, x_test = xs[train_index], xs[test_index]
    y_train, y_test = ys[train_index], ys[test_index]

    # train the classifier
    nbc.fit(x_train,y_train)
    accuracy.append(calculate_accuracy(nbc,x_test,y_test))
    print(f'Accuracy: {calculate_accuracy(nbc,x_test,y_test)}')
print(f'Average accuracy after 3-fold training is {np.array(accuracy).mean()}')

Accuracy: 0.95
Accuracy: 0.9661016949152542
Accuracy: 1.0
Average accuracy after 3-fold training is 0.9720338983050847


### 3. Softmax and Cross Entropy Loss.

(a) I did one PyTorch model without softmax and one PyTorch model with softmax. The output without softmax is a cluster of large positive or negative values. The output with softmax is more like probabilities that sum up to 1.

In [10]:
# First convert the features and labels to PyTorch tensors
pytorch_features = torch.tensor(wines.loc[:, 'Alcohol %':'Proline'].values , dtype=torch.float32)
pytorch_labels = torch.tensor(wines.loc[:, 'ranking'].values, dtype=torch.int64)

# Then define a pytorch model without softmax
model_no_softmax = nn.Sequential(
    nn.Linear(pytorch_features.shape[1], len(np.unique(pytorch_labels)))
)

# Then pass the data through the model once without backpropagation
outputs_no_softmax = model_no_softmax(pytorch_features)

# Finally print out the outputs_no_softmax
print(outputs_no_softmax)

tensor([[ -95.7595,  -80.1433, -271.1181],
        [ -70.3680,  -55.5473, -186.5312],
        [ -89.5526,  -77.3873, -266.7516],
        [-105.8480,  -94.7881, -328.4174],
        [-107.6909,  -97.2809, -338.8018],
        [-109.6877,  -95.2311, -327.6151],
        [ -75.2029,  -59.1946, -197.1850],
        [ -86.9012,  -75.8067, -259.8968],
        [ -74.8449,  -63.2743, -215.8418],
        [-105.9978,  -95.4717, -329.8702],
        [ -88.4649,  -76.9907, -264.4617],
        [-108.5639,  -93.0019, -315.7316],
        [ -79.9209,  -66.1011, -224.1377],
        [ -93.8444,  -82.1973, -283.1660],
        [ -93.6766,  -81.3249, -279.8405],
        [ -79.4933,  -66.3302, -225.2316],
        [ -91.1583,  -78.3610, -270.7823],
        [-104.6139,  -93.3383, -324.4534],
        [-102.0187,  -88.3862, -303.7653],
        [-108.2847,  -95.1919, -328.8627],
        [ -64.1282,  -51.7691, -172.8694],
        [ -43.1073,  -31.5106, -105.4305],
        [ -47.6285,  -37.0139, -126.4444],
        [ -

In [11]:
# Second is to define a pytorch model with softmax
model_softmax = nn.Sequential(
    nn.Linear(pytorch_features.shape[1], len(np.unique(pytorch_labels))),
    nn.Softmax(dim=1)
)

# Then pass the data through the model once without backpropagation
outputs_softmax = model_softmax(pytorch_features)

# Finally print out the outputs_softmax
print(outputs_softmax)

tensor([[1.8472e-04, 0.0000e+00, 9.9982e-01],
        [8.6887e-05, 0.0000e+00, 9.9991e-01],
        [3.2321e-02, 0.0000e+00, 9.6768e-01],
        [5.2833e-01, 0.0000e+00, 4.7167e-01],
        [8.3785e-01, 0.0000e+00, 1.6215e-01],
        [1.6370e-02, 0.0000e+00, 9.8363e-01],
        [8.5346e-06, 0.0000e+00, 9.9999e-01],
        [7.0744e-02, 0.0000e+00, 9.2926e-01],
        [1.5047e-02, 0.0000e+00, 9.8495e-01],
        [6.7305e-01, 0.0000e+00, 3.2695e-01],
        [5.4550e-02, 0.0000e+00, 9.4545e-01],
        [8.9474e-04, 0.0000e+00, 9.9911e-01],
        [4.5460e-04, 0.0000e+00, 9.9955e-01],
        [1.0055e-01, 0.0000e+00, 8.9945e-01],
        [3.4522e-02, 0.0000e+00, 9.6548e-01],
        [1.5719e-03, 0.0000e+00, 9.9843e-01],
        [2.9096e-02, 0.0000e+00, 9.7090e-01],
        [5.5085e-01, 0.0000e+00, 4.4915e-01],
        [1.0845e-02, 0.0000e+00, 9.8915e-01],
        [7.0314e-02, 0.0000e+00, 9.2969e-01],
        [2.5039e-04, 0.0000e+00, 9.9975e-01],
        [3.3493e-04, 8.6922e-32, 9

(b) I unfortunately always encounter an error of dead kernel when trying to evaluate the train_and_val function. I don't really understand the root cause of the issue.

In [14]:
def train_and_val(model,train_X,train_y,epochs,draw_curve=True):
    """
    Parameters
    --------------
    model: a PyTorch model
    train_X: np.array shape(ndata,nfeatures)
    train_y: np.array shape(ndata)
    epochs: int
    draw_curve: bool
    """
    
    ### Define your loss function, optimizer. Convert data to torch tensor ###
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr = 0.001)
    
    train_y -= 1
    Xs = torch.tensor(train_X).float()
    ys = torch.tensor(train_y).long()
    
    # Define Kfolds 
    kf = KFold(n_splits = 3,shuffle = True)
    for train_index, test_index in kf.split(Xs):
        train_X, test_X = Xs[train_index], Xs[test_index]
        train_y, test_y = ys[train_index], ys[test_index]
    
    ### Split training examples further into training and validation ###
    train_X,val_X,train_y,val_y = train_test_split(train_X,train_y, test_size = 0.20)
    val_array=[]
    lowest_val_loss = np.inf
    
    for i in range(epochs):
        ### Compute the loss and do backpropagation ###
        optimizer.zero_grad()
        train_out = model(train_X)
        train_loss = loss(train_out, train_y)
        train_loss.backward()
        optimizer.step()
        
        ### compute validation loss and keep track of the lowest val loss ###
        # compute validation loss
        val_out = model(val_X) 
        val_loss = loss(val_out, val_y)
        
        # append val loss to val_array
        val_array.append(val_loss.item())

        # keep track of the lowest val loss
        if val_loss < lowest_val_loss:
            lowest_val_loss = val_loss
            torch.save(model.state_dict(), 'model.pt')
  
    # The final number of epochs is when the minimum error in validation set occurs    
   
    final_epochs = np.argmin(val_array) + 1
    print("Number of epochs with lowest validation:",final_epochs)
    ### Recover the model weight ###
    model.load_state_dict(torch.load('model.pt'))
    model.eval()

    ### Plot the validation loss curve ###

    if draw_curve:
        plt.figure()
        plt.plot(np.arange(len(val_array))+1,val_array,label='Validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

In [ ]:
train_and_val(model_softmax,pytorch_features,pytorch_labels,1000,draw_curve=True)